In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import time
from sklearn import *

from itertools import product
# from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import cross_val_score, StratifiedKFold



In [2]:
#Load Data into Dataframe for Stacking 
# Load the training data
df_X = pd.read_table('data/X_train.txt', delim_whitespace=True, names=None,header=None)
df_Y = pd.read_table('data/Y_train.txt', delim_whitespace=True, names=None,header=None)

# And the test features
df_Xte = pd.read_table('data/X_test.txt', delim_whitespace=True, names=None,header=None)


In [3]:
#Check length of test data
df_Xte.shape[0]

200000

In [4]:
#Check if data has been loaded correctly 
df_X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,242.0,227.0,240.92,232.44,1195.0,253.0,0.0,1.26710,6.4128,1.98690,3.9756,2.3392,6.3537,0.0
1,249.0,230.0,242.31,233.68,1579.0,243.0,0.0,9.68310,6.0824,1.19640,3.4577,2.0416,7.6746,0.0
2,223.0,195.0,227.64,204.42,1034.0,603.0,318.0,1.52860,17.8690,13.23000,5.7120,4.7216,6.6030,0.0
3,234.0,221.0,236.27,229.73,7716.0,3907.0,0.0,0.60465,8.0497,3.44760,2.4845,1.5741,1.4205,0.0
4,234.0,233.0,245.51,234.10,545.0,21.0,0.0,6.74730,5.2649,0.86766,5.9626,4.2450,3.1429,24.2


In [5]:
#Check if data has been loaded correctly 
df_Y.head()

,0
0,0
1,1
2,1
3,1
4,0


In [6]:
#Check Test data load
df_Xte.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,246.00,226.0,243.27,233.43,7121.0,648.0,0.0,1.1235,5.5226,1.9747,2.3390,1.1438,5.3022,0.0
1,239.00,226.0,240.77,233.31,2109.0,560.0,0.0,5.7154,6.3213,1.5999,4.3318,3.1834,3.0579,0.0
2,251.62,232.0,245.82,233.57,199.0,14.0,0.0,3.4131,5.7395,1.2936,5.6142,2.0601,20.0000,0.0
3,253.00,243.0,249.18,249.18,4095.0,0.0,0.0,10.3200,2.5050,0.0000,1.5908,1.2399,20.0000,0.0
4,247.00,240.0,248.12,248.12,321.0,0.0,0.0,10.8170,3.4157,0.0000,4.2366,1.2236,20.0000,0.0


In [7]:
# Load Data in the form of arrays to perform individual Model training & testing
# Load the training data
X = np.genfromtxt('data/X_train.txt', delimiter=None)
Y = np.genfromtxt('data/Y_train.txt', delimiter=None)

# And the test features
Xte = np.genfromtxt('data/X_test.txt', delimiter=None)

In [8]:
#Check Shape
print(X.shape, Y.shape, Xte.shape)

(200000, 14) (200000,) (200000, 14)


In [9]:
#Check loaded test data
Xte

array([[246.    , 226.    , 243.27  , ...,   1.1438,   5.3022,   0.    ],
       [239.    , 226.    , 240.77  , ...,   3.1834,   3.0579,   0.    ],
       [251.62  , 232.    , 245.82  , ...,   2.0601,  20.    ,   0.    ],
       ...,
       [252.    , 237.    , 248.23  , ...,   1.8303,  20.    ,   0.    ],
       [247.    , 235.    , 247.1   , ...,   2.1439,  20.    ,   0.    ],
       [247.    , 238.    , 248.61  , ...,   1.5008,  20.    ,   0.    ]])

In [10]:
#Check X data
X

array([[242.    , 227.    , 240.92  , ...,   2.3392,   6.3537,   0.    ],
       [249.    , 230.    , 242.31  , ...,   2.0416,   7.6746,   0.    ],
       [223.    , 195.    , 227.64  , ...,   4.7216,   6.603 ,   0.    ],
       ...,
       [250.    , 234.    , 244.51  , ...,   3.3698,  20.    ,   0.    ],
       [236.5   , 227.    , 241.98  , ...,   2.7834,   2.5209,   0.    ],
       [242.    , 229.    , 241.38  , ...,   1.9358,   2.1394,   0.    ]])

In [11]:
#Check Y data
Y

array([0., 1., 1., ..., 0., 0., 1.])

In [12]:
# split Array data training / validation data for individual model training
x_train, x_validation, y_train, y_validation = model_selection.train_test_split(
    X, Y, test_size=0.2, random_state=42)

In [13]:
y_validation

array([0., 0., 1., ..., 0., 0., 0.])

### Functions STARTS

In [26]:
#Function to save Individual Model Test Probabilities for Submission to Kaggle
def SavetoTextFile(model_name,probs,Xte):
    # Create the data for submission by taking the P(Y=1) column from probs and just add a running index as the first column.
    Y_sub = np.vstack([np.arange(Xte.shape[0]), probs[:, 1]]).T
    # We specify the header (ID, Prob1) and also specify the comments as '' so the header won't be commented out with
    # the sign.
    file_name='Solutions/Y_sub_'+model_name+'.txt'
    np.savetxt(file_name, Y_sub, '%d, %.5f', header='ID,Prob1', comments='', delimiter=',')
    print('{} Saved'.format(file_name))
    

In [42]:
def auc_calculate_print(x,y,classifier):
    roc = metrics.roc_auc_score(y,classifier.predict_proba(x)[:,1])
    return(roc)

### Functions ENDS

### Ada Boost STARTS

learning_ratefloat, default=1.
- Learning rate shrinks the contribution of each classifier by learning_rate. There is a trade-off between learning_rate and n_estimators.


In [41]:
ada_boost_classifier = AdaBoostClassifier(tree.DecisionTreeClassifier(random_state=0),
                                          algorithm='SAMME.R',n_estimators=200, learning_rate=0.5)

print("Ada Boost training started")

starting_time = time.time()

ada_boost_classifier.fit(x_train, y_train)

end_time = time.time()

print("Ada Boost training finished, took {} seconds".format(end_time - starting_time))

#Validation AUC Calculation

ada_boost_classifier_validation_roc = auc_calculate_print(x_validation,y_validation,ada_boost_classifier)

print('Ada Boost Validation AUC: {}'.format(ada_boost_classifier_validation_roc))

#Training AUC Calculation

ada_boost_classifier_train_roc= auc_calculate_print(x_train,y_train,ada_boost_classifier)

print('Ada Boost Train AUC: {}'.format(ada_boost_classifier_train_roc))

Ada Boost training started
Ada Boost training finished, took 915.8517315387726 seconds
Ada Boost Validation AUC: 0.6599961391510298
Ada Boost Train AUC: 0.9937726770838594


In [44]:
#Getting the individual Probabilities
#This is what needs to be converted to txt for submission

ada_boost_classifier_probs = ada_boost_classifier.predict_proba(Xte)
print(ada_boost_classifier_probs[:5])

[[3.11943906e-03 9.96880561e-01]
 [1.56564617e-01 8.43435383e-01]
 [9.84272850e-01 1.57271498e-02]
 [9.99869853e-01 1.30146850e-04]
 [9.99409280e-01 5.90720119e-04]]


In [45]:
# Make the prediction probabilities text file for Submission to Kaggle
SavetoTextFile('ada_boost',ada_boost_classifier_probs,Xte)

Solutions/Y_sub_ada_boost.txt Saved


In [46]:
#Cross Checking scores
#For reference only
y_pred = ada_boost_classifier.predict(x_validation)

In [47]:
#Cross Checking scores
#For reference only
cm = confusion_matrix(y_validation,y_pred)
print(cm)
accuracy_score(y_validation,y_pred)

[[21354  4937]
 [ 6138  7571]]


0.723125

*** Ada Boost Hyper Parameter Optimization ***

*** n_estimators: ***

- DecisionTreeClassifier(random_state=0),algorithm='SAMME.R',n_estimators=100, learning_rate=0.5  - 0.7277 Validation AUC,  .9937 Train AUC
- DecisionTreeClassifier(random_state=0),algorithm='SAMME.R',n_estimators=200, learning_rate=0.5  - 0.7374 Validation AUC,  .9937 Train AUC - 489 Secs
- DecisionTreeClassifier(random_state=0),algorithm='SAMME.R',n_estimators=500, learning_rate=0.5  - 0.7476 Validation AUC,  .9937 Train AUC - 1067 Secs

Since, the Validation AUC is not improving substantially .7274 (n_estimators=200) Vs 0.7476 (n_estimators=500) we change the learning rate to 0.7 & n_estimators=200

*** learning_rate: ***

- DecisionTreeClassifier(random_state=0),algorithm='SAMME.R',n_estimators=500, learning_rate=0.7  - 0.7390 Validation AUC,  .9937 Train AUC - 1067 Secs

Since, learning rate = 0.7 caused a drop in the AUC accuracy from .7476 (learning_rate=0.5) to  0.7390 (learning_rate=0.7) we decrease the learning rate to 0.6 & check the results

- DecisionTreeClassifier(random_state=0),algorithm='SAMME.R',n_estimators=200, learning_rate=0.6  - 0.7387 Validation AUC,  .9937 Train AUC - 459 Secs
- DecisionTreeClassifier(random_state=0),algorithm='SAMME.R',n_estimators=500, learning_rate=0.6  - 0.7478 Validation AUC,  .9937 Train AUC - 1017 Secs

*** Conclusion: ***

From the data above its clear that higher number of n_estimators or Trees is not making much of a impact on the Validation AUC, hence we will choose:
- n_estimators=200, learning_rate=0.5 
- as its provides the optimum Validation AUC of .7374 in less amount of training time

### Ada Boost ENDS